# Extracción y descripción de los datos

Empiezo importando las librerías que voy a utilizar.

In [2]:
%matplotlib inline

import numpy as np
import pandas as pd

import librosa.display
import librosa

from glob import glob

import ffmpeg

import os

from pydub import AudioSegment
from pydub.utils import make_chunks

import re

Continuo creando 3 funciones con las que voy a hacer los siguiente:
 - La primera divide un archivo audio de larga duración en pequeños audios de 10 segundos de duración, y los guarda en la carpeta que le he especificado.
 - La segunda accede a cada uno de esos audios de 10 segundos y saca las features características de dicho archivo, y en base a su nombre le asigna una categoría y otra.
 - La tercera simplemente me enseña las principales características y métricas de la base de datos.

Finalmente, guardo el el dataset en mi carpeta como un CSV.

Dado que la base de datos ya la tengo creada, este JN lo voy a aplicar sobre audios que luego voy a utilizar para predecir con mi modelo y ver si es lo suficientemente bueno.

In [43]:
def decompose_files(data_dir, audio_files):
    for j in range(len(audio_files)):
        myaudio = AudioSegment.from_file(data_dir + '{}'.format(os.listdir(data_dir)[j])) 
        chunk_length_ms = 2000
        chunks = make_chunks(myaudio, chunk_length_ms)

        for i, chunk in enumerate(chunks):
            chunk_name = "{}{}.wav".format(os.listdir(data_dir)[j], i)
            #print("exporting", chunk_name)
            chunk.export('./four2/{}'.format(chunk_name), format="mp3")
            
decompose_files('./four/', glob('./four' + '/*'))

In [44]:
def get_features(data_dir, audio_files):
    momentos = {'ducha': 'Ducha', 'cena': 'Cena', 'washing': 'Lavadora', 'vacuum': 'Aspiradora', 
                'shaver': 'Afeitadora', 'hairdryer': 'Secador_pelo', 'airconditioner': 'Aire_acondicionado', 
                'cellphone': 'Telefono', 'comp': 'Tecleo', 'silence': 'Silencio', 'dryer': 'Secadora', 
                'blender': 'Licuadora', 'doorbell': 'Timbre', 'alarm': 'Alarma', 'faucet': 'Grifo', 
                'microwave': 'Microondas'}
    features = []
    for i in range(len(audio_files)):
        y, sr = librosa.load(audio_files[i], sr=8000, mono=True)
        name = os.listdir(data_dir)[i]
        pattern = "[._][\w]+"
        name2 = re.sub(pattern, '', name)
        if name2 in momentos.keys():
            momento = momentos.get(name2)
        else:
            momento = 'Otro'
        mfcc = np.ndarray.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20))
        scem = np.ndarray.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
        scom = np.ndarray.mean(librosa.feature.spectral_contrast(S=np.abs(librosa.stft(y)), sr=sr, n_bands=4))
        srom = np.ndarray.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
        sbwm = np.ndarray.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
        tempo = librosa.beat.tempo(onset_envelope=librosa.onset.onset_strength(y=y, sr=sr, hop_length=512), sr=sr, hop_length=512)[0]
        rmse = np.ndarray.mean(librosa.feature.rms(y=y))
        D = np.abs(np.fft.fft(y, n=32)[:8000*2 // 2])
        features.append([name, mfcc, scem, scom, srom, sbwm, tempo, rmse, 
                         D[0], D[1], D[2], D[3], D[4], D[5], 
                         D[6], D[7], D[8], D[9], D[10], D[11], 
                         D[12], D[13], D[14], D[15], D[16], D[17], 
                         D[18], D[19], D[20], D[21], D[22], D[23],
                         D[24], D[25], D[26], D[27], D[28], D[29],
                         D[30], D[31], momento])
    return pd.DataFrame(features, columns=['name', 'mfcc', 'scem','scom', 'srom','sbwm', 'tempo', 'rmse', 
                                           'Fourier1', 'Fourier2', 'Fourier3', 'Fourier4', 'Fourier5', 
                                           'Fourier6', 'Fourier7', 'Fourier8', 'Fourier9', 'Fourier10',
                                           'Fourier11', 'Fourier12', 'Fourier13', 'Fourier14', 'Fourier15',
                                           'Fourier16', 'Fourier17', 'Fourier18', 'Fourier19', 'Fourier20',
                                           'Fourier21', 'Fourier22', 'Fourier23', 'Fourier24', 'Fourier25',
                                           'Fourier26', 'Fourier27', 'Fourier28', 'Fourier29', 'Fourier30',
                                           'Fourier31', 'Fourier32', 'momento'])

data2 = get_features('./four2/', glob('./four2' + '/*.wav'))

/home/linuxbrew/.linuxbrew/opt/python/lib/python3.7/site-packages/librosa/filters.py:284: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '
/home/linuxbrew/.linuxbrew/opt/python/lib/python3.7/site-packages/librosa/filters.py:284: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '
/home/linuxbrew/.linuxbrew/opt/python/lib/python3.7/site-packages/librosa/filters.py:284: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '
/home/linuxbrew/.linuxbre

In [45]:
data2.shape

(18, 41)

In [46]:
data2[data2['Fourier5'] == 0].shape

(2, 41)

In [47]:
data_alt2 = data2[data2['Fourier5'] != 0]

In [48]:
data_alt3 = data_alt2.drop(['name'], axis=1)

In [49]:
data_alt3[data_alt3['Fourier5'] == 0]

,mfcc,scem,scom,srom,sbwm,tempo,rmse,Fourier1,Fourier2,Fourier3,...,Fourier24,Fourier25,Fourier26,Fourier27,Fourier28,Fourier29,Fourier30,Fourier31,Fourier32,momento


In [50]:
data.describe()

,mfcc,scem,scom,srom,sbwm,tempo,rmse,Fourier1,Fourier2,Fourier3,Fourier4,Fourier5,Fourier6,Fourier7,Fourier8,Fourier9,Fourier10
count,451.000000,451.000000,451.000000,451.000000,451.000000,451.000000,451.000000,451.000000,451.000000,451.000000,451.000000,451.000000,451.000000,451.000000,451.000000,451.000000,451.000000
mean,-10.312899,4008.277586,20.954905,7387.158317,2795.017846,116.167294,0.109255,0.344227,0.330436,0.182743,0.183696,0.114038,0.030733,0.114038,0.183696,0.182743,0.330436
std,4.060308,290.019242,0.885593,528.644235,211.138154,14.136493,0.017034,0.435237,0.394935,0.229203,0.216392,0.128409,0.047401,0.128409,0.216392,0.229203,0.394935
min,-23.105433,2947.191723,17.991425,5329.096511,1977.107995,36.914062,0.055766,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-9.656688,4023.197301,20.939618,7503.207503,2853.216105,117.453835,0.115653,0.019292,0.005926,0.003811,0.004697,0.003393,0.001612,0.003393,0.004697,0.003811,0.005926
50%,-9.172000,4104.063456,21.203728,7545.655139,2869.987450,117.453835,0.116717,0.063875,0.014274,0.008655,0.011680,0.007180,0.005224,0.007180,0.011680,0.008655,0.014274
75%,-8.541408,4158.967848,21.487418,7618.855654,2878.461322,117.453835,0.117617,0.568106,0.664518,0.354261,0.374597,0.235738,0.043737,0.235738,0.374597,0.354261,0.664518
max,-5.895563,4229.816411,22.300036,7839.694740,2982.233360,198.768029,0.119760,1.477407,1.220708,0.844557,0.685030,0.411593,0.239982,0.411593,0.685030,0.844557,1.220708


In [51]:
def show_df_info(dataframe):
    display("The type of the dataframe is {}.".format(type(dataframe)))
    print("------------------------------------------------------------------------")
    display("The amount of entries are {}.".format(dataframe.size))
    print("------------------------------------------------------------------------")
    display("Its shape is {}.".format(dataframe.shape))
    print("------------------------------------------------------------------------")
    display("Its features are: {}.".format(list(dataframe)))
    print("------------------------------------------------------------------------")
    print("The data types of columns are: {}".format(dataframe.dtypes))
    print("------------------------------------------------------------------------")
    print("The counter of each class is: {}".format(dataframe.momento.value_counts()))
    print("------------------------------------------------------------------------")
    print('The amount of null values is: {}.'.format(dataframe.isna().sum()))
    display(dataframe.head())
    display(dataframe.describe())
    
show_df_info(data_alt3)

"The type of the dataframe is <class 'pandas.core.frame.DataFrame'>."

------------------------------------------------------------------------


'The amount of entries are 640.'

------------------------------------------------------------------------


'Its shape is (16, 40).'

------------------------------------------------------------------------


"Its features are: ['mfcc', 'scem', 'scom', 'srom', 'sbwm', 'tempo', 'rmse', 'Fourier1', 'Fourier2', 'Fourier3', 'Fourier4', 'Fourier5', 'Fourier6', 'Fourier7', 'Fourier8', 'Fourier9', 'Fourier10', 'Fourier11', 'Fourier12', 'Fourier13', 'Fourier14', 'Fourier15', 'Fourier16', 'Fourier17', 'Fourier18', 'Fourier19', 'Fourier20', 'Fourier21', 'Fourier22', 'Fourier23', 'Fourier24', 'Fourier25', 'Fourier26', 'Fourier27', 'Fourier28', 'Fourier29', 'Fourier30', 'Fourier31', 'Fourier32', 'momento']."

------------------------------------------------------------------------
The data types of columns are: mfcc         float64
scem         float64
scom         float64
srom         float64
sbwm         float64
tempo        float64
rmse         float64
Fourier1     float64
Fourier2     float64
Fourier3     float64
Fourier4     float64
Fourier5     float64
Fourier6     float64
Fourier7     float64
Fourier8     float64
Fourier9     float64
Fourier10    float64
Fourier11    float64
Fourier12    float64
Fourier13    float64
Fourier14    float64
Fourier15    float64
Fourier16    float64
Fourier17    float64
Fourier18    float64
Fourier19    float64
Fourier20    float64
Fourier21    float64
Fourier22    float64
Fourier23    float64
Fourier24    float64
Fourier25    float64
Fourier26    float64
Fourier27    float64
Fourier28    float64
Fourier29    float64
Fourier30    float64
Fourier31    float64
Fourier32    float64
momento       object
dtype: object
------------------------------------------

,mfcc,scem,scom,srom,sbwm,tempo,rmse,Fourier1,Fourier2,Fourier3,...,Fourier24,Fourier25,Fourier26,Fourier27,Fourier28,Fourier29,Fourier30,Fourier31,Fourier32,momento
0,-1.628756,1414.572244,22.675694,2688.964844,1012.104539,133.928571,0.147794,0.455576,0.855517,1.217035,...,0.227303,0.221763,0.201087,0.402851,0.266518,0.188775,0.448843,1.217035,0.855517,Aspiradora
1,-1.506035,1381.930099,22.745829,2651.977539,1012.829229,133.928571,0.143885,1.337814,0.821697,2.255564,...,0.072751,0.030290,0.495613,0.288902,0.457856,0.366302,0.719223,2.255564,0.821697,Aspiradora
2,-10.171223,1899.130782,22.926630,3330.078125,1053.184961,133.928571,0.166528,0.146833,0.296734,1.424936,...,0.606584,1.065522,0.388100,1.458757,0.561210,0.724556,1.026013,1.424936,0.296734,Aspiradora
3,-6.319030,2068.097241,19.717858,3147.338867,1034.022848,133.928571,0.063924,0.217815,0.299719,0.650947,...,0.303021,0.250098,0.341444,0.131757,0.194542,0.498210,0.393564,0.650947,0.299719,Licuadora
4,-4.020776,1735.879747,32.736242,3238.281250,1069.616398,120.000000,0.178995,0.150675,0.519349,3.256600,...,0.224028,0.152183,0.879043,1.191371,1.137622,1.196225,0.238380,3.256600,0.519349,Aspiradora


,mfcc,scem,scom,srom,sbwm,tempo,rmse,Fourier1,Fourier2,Fourier3,...,Fourier23,Fourier24,Fourier25,Fourier26,Fourier27,Fourier28,Fourier29,Fourier30,Fourier31,Fourier32
count,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,...,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000
mean,-6.311673,1679.191915,23.996033,3003.219604,1037.049069,129.224330,0.122693,0.250486,0.544757,1.553666,...,0.348567,0.304820,0.267382,0.333114,0.560146,0.482503,0.524448,0.490069,1.553666,0.544757
std,3.971102,284.013058,4.069932,304.705027,17.679691,7.250122,0.057191,0.318991,0.475536,1.223761,...,0.333846,0.240193,0.270346,0.274062,0.557369,0.448272,0.406744,0.324991,1.223761,0.475536
min,-12.186146,1208.335661,19.717858,2460.937500,1009.887721,117.187500,0.011367,0.000032,0.000022,0.000042,...,0.000031,0.000011,0.000062,0.000057,0.000035,0.000064,0.000149,0.000078,0.000042,0.000022
25%,-9.749711,1406.411708,22.475199,2701.232910,1025.039952,120.000000,0.065757,0.088562,0.137640,0.507608,...,0.153504,0.099372,0.095756,0.105445,0.122865,0.195503,0.213894,0.314787,0.507608,0.137640
50%,-6.863251,1800.554532,22.964360,3118.652344,1037.065050,133.928571,0.145840,0.148754,0.409534,1.320985,...,0.241597,0.265162,0.222561,0.299034,0.375566,0.446965,0.468065,0.457993,1.320985,0.409534
75%,-2.345912,1886.357204,23.298628,3298.919678,1048.610242,133.928571,0.168432,0.286731,0.872622,2.728294,...,0.465928,0.448413,0.360869,0.460711,0.893398,0.586453,0.742924,0.694825,2.728294,0.872622
max,-1.279333,2068.097241,32.736242,3330.078125,1069.616398,133.928571,0.179560,1.337814,1.495235,3.256600,...,1.145785,0.735094,1.065522,0.879043,1.796916,1.770227,1.254831,1.146715,3.256600,1.495235


In [22]:
#data.to_csv('data_prueba.csv', index=False)

In [52]:
#data_alt3.to_csv('data_fourier_aux.csv', index=False)